# Importando bibliotecas

In [105]:
import pandas  as pd
import numpy   as np

from datetime                import datetime
from sklearn.model_selection import train_test_split
from sklearn.svm             import LinearSVC
from sklearn.metrics         import accuracy_score
from sklearn.dummy           import DummyClassifier

# Importando os dados

In [106]:
uri = 'https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv'
dados = pd.read_csv(uri)
dados.head()

,Unnamed: 0,mileage_per_year,model_year,price,sold
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


# Conhecendo o dataset

Trata-se de um banco de dados fictício em que cada linha representa um carro à venda em uma loja online. Os carros são de diversos donos diferentes, e alguns desses carros foram vendidos e outros não.

**Colunas:**

- sold - carros que foram ou não vendidos, sendo: "yes" para os vendidos e "no" para os não.

- milageperyear - representa a quantidade de milhas o carro andou por ano

- modelyear - representa o ano de cada modelo (que é diferente do ano de fabricação)

- price - preço de venda cada carro.

De acordo com estas variáveis, tentaremos classificar se um carro será vendido ou não.

Pensando no negócio da empresa, com esse tipo de informação, ela poderia ajudar seus clientes a concretizar sua venda, criando condições mais atrativas para possíveis compradores com base em dados reais e de acordo com as características de cada carro, usando o preço como variável decisiva para a venda do carro.

# Tratando os dados

### Renomeando as colunas do dadaset

In [107]:
mapa = { 
    'mileage_per_year' : 'milhas_por_ano',
    'model_year' : 'ano_do_modelo',
    'price' : 'preco',
    'sold' :'vendido'
}

dados = dados.rename(columns = mapa)
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


### Renomenando as opções da coluna 'vendido'

In [108]:
mapa = {
    'no': 0,
    'yes': 1
}
dados.vendido = dados.vendido.map(mapa)
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,1
1,1,7843,1998,40557.96,1
2,2,7109,2006,89627.50,0
3,3,26823,2015,95276.14,0
4,4,7935,2014,117384.68,1


### Criando uma coluna com a idade do modelo

In [109]:
ano_atual = datetime.today().year
dados['idade_do_modelo'] = ano_atual - dados.ano_do_modelo
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo
0,0,21801,2000,30941.02,1,23
1,1,7843,1998,40557.96,1,25
2,2,7109,2006,89627.50,0,17
3,3,26823,2015,95276.14,0,8
4,4,7935,2014,117384.68,1,9


### Criando uma coluna km por ano

In [110]:
#1 milha = 1.60934
dados ['km_por_ano'] = dados.milhas_por_ano * 1.60934
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo,km_por_ano
0,0,21801,2000,30941.02,1,23,35085.22134
1,1,7843,1998,40557.96,1,25,12622.05362
2,2,7109,2006,89627.50,0,17,11440.79806
3,3,26823,2015,95276.14,0,8,43167.32682
4,4,7935,2014,117384.68,1,9,12770.11290


### Retirando do dataframe as colunas : Unnamed, milhas_por_ano e ano_do_modelo

In [111]:
dados = dados.drop(columns = ["Unnamed: 0", 
                              "milhas_por_ano", 
                              "ano_do_modelo"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,23,35085.22134
1,40557.96,1,25,12622.05362
2,89627.50,0,17,11440.79806
3,95276.14,0,8,43167.32682
4,117384.68,1,9,12770.11290


# Modelo: LinearSVC

### Separando as variáveis explicatórias(x) e dependente(y)

In [112]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

### Separando dados de treino e teste

In [113]:
SEED = 20
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, 
                                                        test_size = 0.25,
                                                         stratify = y)

print(f'Treino {treino_x.shape}')
print(f'Teste {teste_x.shape}')

Treino (7500, 3)
Teste (2500, 3)


### Treinando o modelo

In [114]:
modelo = LinearSVC()
modelo.fit(treino_x, treino_y)

/Users/fernanda/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

### Testando o modelo

In [115]:
previsoes = modelo.predict(teste_x)

### Acurácia do modelo

In [116]:
acuracia = accuracy_score(teste_y, previsoes) * 100
print(f'O modelo foi treinado com {len(treino_x)} elementos, testado com {len(teste_x)} e a acurácia do modelo LinearSVC é de: {acuracia.round(2)} %.')

O modelo foi treinado com 7500 elementos, testado com 2500 e a acurácia do modelo LinearSVC é de: 57.2 %.


# Modelo: Dummy

### Treinando o modelo

In [117]:
dummy_stratified = DummyClassifier(strategy='stratified')
dummy_stratified.fit(treino_x, treino_y)

DummyClassifier(strategy='stratified')

### Testando o modelo

In [118]:
previsoes = dummy_stratified.predict(teste_x)

### Acurácia do modelo

In [119]:
acuracia = accuracy_score(teste_y, previsoes) * 100
print(f'O modelo foi treinado com {len(treino_x)} elementos, testado com {len(teste_x)} e a acurácia do modelo Dummy é de: {acuracia.round(2)} %.')

O modelo foi treinado com 7500 elementos, testado com 2500 e a acurácia do modelo Dummy é de: 52.2 %.
